In [26]:
!pip install pydub
!pip install torch-audiomentations

In [27]:
import librosa
import matplotlib.pyplot as plt
import numpy as np
import math
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift
import torch
import librosa
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import matplotlib.pyplot as plt
import numpy as np
import math

/Users/abhkhare/miniconda3/envs/ptENV3/lib/python3.8/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [28]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

/Users/abhkhare/miniconda3/envs/ptENV3/lib/python3.8/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:356: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
audio, rate = librosa.load("../data/ATIS/raw/test/ASR_error/voice/8.aiff", sr = 16000)
input_values = tokenizer(audio, return_tensors = "pt").input_values
logits = model(input_values).logits
prediction = torch.argmax(logits, dim = -1)
transcription = tokenizer.batch_decode(prediction)[0]
print(transcription)

AFTER TWELVE P M ON WEDNESDAY APRIL SIXTH I WOULD LIKE TO FLY FROM LONG BEACH TO COLUMBUS


In [56]:
SAMPLE_RATE = 16000

augment = Compose([
    AddGaussianNoise(min_amplitude=0.005, max_amplitude=0.025, p=0.5),
    TimeStretch(min_rate=0.9, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-3, max_semitones=4, p=0.5),
    Shift(min_fraction=-0.5, max_fraction=0.5, p=0.5),
])

audio, rate = librosa.load("../data/ATIS/raw/test/ASR_error/voice/8.aiff", sr = 16000)

# Augment/transform/perturb the audio data
augmented_samples = augment(samples=audio, sample_rate=SAMPLE_RATE)

In [57]:
input_values = tokenizer(augmented_samples, return_tensors = "pt").input_values
logits = model(input_values).logits
prediction = torch.argmax(logits, dim = -1)
transcription = tokenizer.batch_decode(prediction)[0]
print(transcription)

AFTER GOLV P N ON MONDAY APRIL SIXH I WOULD LIKE TO BUY FROM LONG BEACH I COLAMUS
